<a href="https://colab.research.google.com/github/txusser/Master_IA_Sanidad/blob/main/Modulo_2/2_3_4_Proyecto_Regresion_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, our goal is to build a simple linear regression model using sci-kit learn to predict medical costs based on the dataset available [at this link](https://www.kaggle.com/mirichoi0218/insurance).

The first step will be to upload the data file to the Google virtual machine.


## Load Basic Libraries and Data

In [15]:
# First, we import the basic working libraries for any machine learning project
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from rich.console import Console
console = Console()

# And we load the data contained in our dataset (downloaded and uploaded)
df = pd.read_csv("insurance.csv")
console.log(f"DataFrame Header: {df.head()}", style="bold yellow")

## Data Exploration
As we have seen in Topic 2.3.3, one of the first steps we must take in any Machine Learning project is the descriptive and visual exploration of the working data. This process is essential to thoroughly understand the nature of the data we are working with and to lay the foundation upon which subsequent models will be built.


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from rich.console import Console

console = Console()

# We observe how the target variable "charges" is distributed
# which represents the insurance charges
sns.histplot(df['charges'], stat="density")
plt.title('Distribution of Insurance Charges')
plt.xlabel('Charges')
plt.ylabel('Density')

# Fit the data to a normal distribution
mu, std = norm.fit(df['charges'])

# Plot the PDF (Probability Density Function)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)

plt.plot(x, p, 'k', linewidth=2, color='red')
plt.legend(['Normal Distribution', 'Observed Data'])

# Show the plot
plt.show()

# Print relevant statistics using rich.Console
console.rule("Relevant Statistics")
console.print(f"Mean of Charges: [bold]{mu:.2f}[/bold]")
console.print(f"Standard Deviation of Charges: [bold]{std:.2f}[/bold]")


The Shapiro-Wilk test is a statistical test used to assess whether a dataset follows a normal distribution. The normal distribution, also known as the Gaussian distribution, is a continuous distribution commonly observed in many natural phenomena and serves as a fundamental basis for many statistical methods and tests.

In [7]:
# Define a function to perform a normality test and check
# the Gaussian nature (or not) of the distribution

from scipy.stats import norm, shapiro

def test_normality(data):
    stat, p_value = shapiro(data)
    console.rule("Normality Test")
    console.print(f"Test Statistic: [bold]{stat:.4f}[/bold]")
    console.print(f"P-value: [bold]{p_value:.4f}[/bold]")

    if p_value > 0.05:
        console.print("The data follows a normal distribution (p > 0.05).")
    else:
        console.print("The data does not follow a normal distribution (p <= 0.05).")

# Perform the normality test
test_normality(df['charges'])


From the previous results, we can conclude that the target variable "charges" does not follow a normal distribution but rather a mixed distribution. This could present a challenge in achieving optimal performance for our linear model.

In [9]:
# Reviewing the correlation matrix to identify possible dependencies
correlation_matrix = df.corr()  # Using the method implemented by Pandas

# Create a figure and axis for the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

# Print the correlation of features with "charges," sorted by the highest correlation values
console.rule("Features Most Correlated with 'charges'")
charges_correlation = correlation_matrix['charges'].sort_values(ascending=False)
console.print(charges_correlation)

# Another way to generate the heatmap using Seaborn:
# sns.heatmap(df.corr(), annot=True, cmap='viridis')

In [10]:
# Create a pair plot to visualize the morphology of correlations
sns.set(style="ticks")
pair_plot = sns.pairplot(df)
plt.suptitle("Pair Plot of Features")
plt.show()

# Print a brief description of the pair plot
console.rule("Pair Plot Description")
console.print("The pair plot provides a graphical representation of the relationships between the dataset's features.")
console.print("The diagonal contains individual distributions of the features.")
console.print("The off-diagonal cells display scatter plots between pairs of features, helping identify correlation patterns and trends.")

## Encoding Categorical Variables
The next step is to encode categorical variables.

In [16]:
import pandas as pd

# Categorical variables to encode
vars_to_encode = ['sex', 'smoker', 'region']

# Use the get_dummies method to encode categorical variables
dummies = [pd.get_dummies(df[var], prefix=var) for var in vars_to_encode]
df_dummies = pd.concat(dummies, axis=1)

# Remove the original columns of the categorical variables
df = df.drop(vars_to_encode, axis=1)

# Concatenate the encoded columns with the original DataFrame
df_encoded = pd.concat([df, df_dummies], axis=1)

# Rename columns for better clarity
df_encoded.rename(columns={'smoker_no': 'non-smoker', 'smoker_yes': 'nicotine-user'}, inplace=True)

# Display the updated dataset with encoded variables
console.log(f"Encoded DataFrame: {df_encoded.head()}", style="bold yellow")


## Data Normalization
The next phase of preprocessing involves rescaling the data. In this case, we will apply methods from the StandardScaler class in Sci-kit learn.

In [18]:
from sklearn.preprocessing import StandardScaler

# Create a copy of the encoded DataFrame for scaling
df_c_scaled = df_encoded.copy()

# Use StandardScaler to scale the dataset
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_c_scaled)
df_c_scaled[df_c_scaled.columns] = scaled_features

# Display the scaled dataset
print("\n Scaled Dataset: \n", df_c_scaled)

# Visualize the distribution of the "charges" column in the scaled dataset
sns.histplot(df_c_scaled["charges"])
plt.title("Distribution of Scaled Charges")
plt.xlabel("Scaled Charges")
plt.ylabel("Frequency")
plt.show()

## Feature Extraction

In [19]:
# We will apply the PCA technique to identify the most representative variables
from sklearn.decomposition import PCA
df_s = df_c_scaled  # Our previously processed working DataFrame

# The feature names are identified in the column headers
features = df_s.columns
X = df_s[features]

# Analyze the complete set of variables
pca = PCA(n_components=len(features), random_state=2020)
pca.fit(X)
X_pca = pca.transform(X)

# Plot the percentage of variance explained vs the number of components
plt.plot(np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel("Number of Components")
plt.ylabel("Percentage of Variance Explained")

In [23]:
# Create an instance of PCA with 6 principal components
pca_s = PCA(n_components=6, random_state=2020)

# Fit the PCA model to the original data
pca_s.fit(X)

# Transform the original data using the principal components
X_pca_s = pca_s.transform(X)

# Create a new DataFrame with the transformed data
cols = ['PCA' + str(i) for i in range(1, 7)]  # Changed to make components range from 1 to 6
df_pca = pd.DataFrame(X_pca_s, columns=cols)

# Finally, observe the dataset transformed with the 6 principal components
console.log(f"Data for the 6 principal components:{df_pca}", style="bold yellow")


## Linear Model Fitting
After completing the data preparation process, we will proceed to fit the linear model.


In [26]:
from sklearn.model_selection import train_test_split

# The features we are analyzing are those selected with PCA
pca_features = df_pca.columns
X = df_pca[pca_features]
y = df_s['charges']

# Use the train_test_split function from sklearn to split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

print("Training and testing set sizes:")
print(" - X_train:", X_train.shape)
print(" - X_test:", X_test.shape)
print(" - y_train:", y_train.shape)
print(" - y_test:", y_test.shape)

In [27]:
# Expose the training set to the linear regression model

from sklearn.linear_model import LinearRegression

# Create an instance of the linear regression model
lm = LinearRegression()

# Fit the model to the training data
lm.fit(X_train, y_train)


In [29]:
from sklearn import metrics
import numpy as np

# Calculate the model's predictions on the test set
y_pred = lm.predict(X_test)

# Compute model fit and performance metrics
evar = metrics.explained_variance_score(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Finally, print the fit and performance metrics
print("Fit and Performance Metrics:")
print('- Explained Variance: ', round(evar, 2))
print('- R2:', round(r2, 2))
print('- MAE: ', round(mae, 4))
print('- MSE: ', round(mse, 4))
print('- RMSE: ', round(rmse, 4))


In [30]:
# To finish, we represent the model's predictions in a graph:

# Scatter plot of predictions vs. actual values
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(y_pred, y_test, color='blue', edgecolors=(0, 0, 1), label='Test Data')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3, label='45° Line')

# Labels and title of the graph
ax.set_xlabel('Prediction')
ax.set_ylabel('Actual Value')
ax.set_title('Comparison of Predictions and Actual Values')
ax.legend()
plt.show()
